In [1]:
!pip freeze | grep scikit-learn

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
import pickle
import pandas as pd
import numpy as np

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    
    return df

In [5]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

Q1

In [9]:
np.std(y_pred)

5.28140357655334

In [11]:
year = 2022
month = int('02')


df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [13]:
df_result = pd.DataFrame()

df_result['ride_id'] = df['ride_id']
df_result['predictions'] = y_pred

In [18]:
output_file = "./output/yellow_taxi_predictions.parquet"

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)